In [1]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import matplotlib.font_manager as fm

In [3]:
if 'google.colab' in sys.modules:
  !echo 'debconf debconf/frontend select Noninteractive' | \
  debconf-set-selections

  !sudo apt-get -qq -y install fonts-nanum
  fm._rebuild()

Selecting previously unselected package fonts-nanum.
(Reading database ... 129496 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20180306-3_all.deb ...
Unpacking fonts-nanum (20180306-3) ...
Setting up fonts-nanum (20180306-3) ...
Processing triggers for fontconfig (2.13.1-2ubuntu3) ...


In [3]:
fm.findSystemFonts()

['/usr/share/fonts/truetype/liberation/LiberationMono-Italic.ttf',
 '/usr/share/fonts/truetype/nanum/NanumSquareRoundB.ttf',
 '/usr/share/fonts/truetype/nanum/NanumMyeongjoBold.ttf',
 '/usr/share/fonts/truetype/humor-sans/Humor-Sans.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationSans-BoldItalic.ttf',
 '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationMono-Regular.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationSerif-Bold.ttf',
 '/usr/share/fonts/truetype/nanum/NanumGothicBold.ttf',
 '/usr/share/fonts/truetype/nanum/NanumSquareRoundR.ttf',
 '/usr/share/fonts/truetype/nanum/NanumBarunGothicBold.ttf',
 '/usr/share/fonts/truetype/nanum/NanumGothic.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationSansNarrow-BoldItalic.ttf',
 '/usr/share/fonts/truetype/liberation/LiberationSerif-BoldItalic.ttf',
 '/usr/share/fonts/truetype/nanum/NanumSquareB.ttf',
 '

In [5]:
plt.rc('font', family='NanuBarunGothic', size=10)
plt.rcParams['figure.dpi'] = 100

In [12]:
conn = sqlite3.connect('/content/drive/MyDrive/Colab Notebooks/classicmodels.db')

In [13]:
# 일별 매출액

day_rev = pd.read_sql_query(
    "SELECT A.ORDERDATE, SUM(PRICEEACH*QUANTITYORDERED) AS SALES\
    FROM ORDERS A\
    LEFT JOIN ORDERDETAILS B\
    ON A.ORDERNUMBER = B.ORDERNUMBER\
    GROUP BY 1\
    ORDER BY 1;"
    , conn)

day_rev.head()

,orderDate,SALES
0,2003-01-06 00:00:00,10223.83
1,2003-01-09 00:00:00,10549.01
2,2003-01-10 00:00:00,5494.78
3,2003-01-29 00:00:00,50218.95
4,2003-01-31 00:00:00,40206.20


In [15]:
# 월별 매출액

month_rev = pd.read_sql_query(
    "SELECT\
    SUBSTR(A.ORDERDATE, 1, 7) MM,\
    SUM(PRICEEACH* QUANTITYORDERED) AS M_SALES\
    FROM ORDERS A\
    LEFT JOIN ORDERDETAILS B\
    ON A.ORDERNUMBER = B.ORDERNUMBER\
    GROUP BY 1\
    ORDER BY 1;"
    , conn)

month_rev.head()

,MM,M_SALES
0,2003-01,116692.77
1,2003-02,128403.64
2,2003-03,160517.14
3,2003-04,185848.59
4,2003-05,179435.55


In [17]:
# 연도별 매출액

year_rev = pd.read_sql_query(
    "SELECT\
    SUBSTR(A.ORDERDATE, 1,4) YY,\
    SUM(PRICEEACH*QUANTITYORDERED) AS Y_SALES\
    FROM ORDERS A\
    LEFT JOIN ORDERDETAILS B\
    ON A.ORDERNUMBER = B.ORDERNUMBER\
    GROUP BY 1\
    ORDER BY 1;",
    conn)

year_rev.head()

,YY,Y_SALES
0,2003,3317348.39
1,2004,4515905.51
2,2005,1770936.71


In [20]:
# 일자별 구매자 수, 구매 건수

day_ctm = pd.read_sql_query(
    "SELECT ORDERDATE,\
    COUNT(DISTINCT CUSTOMERNUMBER) N_PURCHASER,\
    COUNT(ORDERNUMBER) N_ORDERS\
    FROM ORDERS\
    GROUP BY 1\
    ORDER BY 2 DESC;",
    conn)
day_ctm.head()

,orderDate,N_PURCHASER,N_ORDERS
0,2004-11-24 00:00:00,4,4
1,2004-11-05 00:00:00,3,3
2,2003-12-02 00:00:00,3,3
3,2003-12-01 00:00:00,3,3
4,2003-11-14 00:00:00,3,3


In [33]:
# 월별 구매자 수, 구매 건수

month_ctm = pd.read_sql_query(
    "SELECT SUBSTR(ORDERDATE,1,7) MM,\
    COUNT(DISTINCT CUSTOMERNUMBER) N_PURCHASER,\
    COUNT(ORDERNUMBER) N_ORDERS\
    FROM ORDERS\
    GROUP BY 1\
    ORDER BY 1;",
    conn)

month_ctm.head()

,MM,N_PURCHASER,N_ORDERS
0,2003-01,5,5
1,2003-02,3,3
2,2003-03,6,6
3,2003-04,7,7
4,2003-05,6,6


In [35]:
# 연도별 구매자 수, 구매 건수

year_ctm = pd.read_sql_query(
    "SELECT SUBSTR(ORDERDATE,1,4) YY,\
    COUNT(DISTINCT CUSTOMERNUMBER) N_PURCHASER,\
    COUNT(ORDERNUMBER) N_ORDERS\
    FROM ORDERS\
    GROUP BY 1\
    ORDER BY 1;",
    conn)

year_ctm.head()

,YY,N_PURCHASER,N_ORDERS
0,2003,74,111
1,2004,89,151
2,2005,44,64


In [38]:
# 연도별 1인당 구매액 AMV

per_year_ctm = pd.read_sql_query(
    "SELECT SUBSTR(ORDERDATE,1,4) YY,\
    COUNT(DISTINCT A.CUSTOMERNUMBER) N_PURCHASER,\
    SUM(PRICEEACH * QUANTITYORDERED) AS SALES,\
    SUM(PRICEEACH * QUANTITYORDERED) / COUNT(DISTINCT A.CUSTOMERNUMBER) AMV\
    FROM ORDERS A\
    LEFT JOIN ORDERDETAILS B\
    ON A.ORDERNUMBER = B.ORDERNUMBER\
    GROUP BY 1\
    ORDER BY 1;",
    conn)
per_year_ctm.head() 

,YY,N_PURCHASER,SALES,AMV
0,2003,74,3317348.39,44829.032297
1,2004,89,4515905.51,50740.511348
2,2005,44,1770936.71,40248.561591


In [40]:
# 연도별 주문당 구매액 ATV

per_year_order = pd.read_sql_query(
    "SELECT SUBSTR(A.ORDERDATE,1,4) YY,\
    COUNT(DISTINCT A.CUSTOMERNUMBER) N_PURCHASER,\
    SUM(PRICEEACH*QUANTITYORDERED) AS SALES,\
    SUM(PRICEEACH*QUANTITYORDERED) / COUNT(DISTINCT A.ORDERNUMBER) AS ATV\
    FROM ORDERS A\
    LEFT JOIN ORDERDETAILS B\
    ON A.ORDERNUMBER = B.ORDERNUMBER\
    GROUP BY 1\
    ORDER BY 1;",
    conn)

per_year_order.head()

,YY,N_PURCHASER,SALES,ATV
0,2003,74,3317348.39,29886.021532
1,2004,89,4515905.51,29906.659007
2,2005,44,1770936.71,27670.886094


# 그룹별 지표 확인

In [41]:
# 국가별 도시별 매출액  

group_basic_df = pd.read_sql_query(
    "SELECT *\
    FROM ORDERS A\
    LEFT JOIN ORDERDETAILS B\
    ON A.ORDERNUMBER = B.ORDERNUMBER\
    LEFT JOIN CUSTOMERS C\
    ON A.CUSTOMERNUMBER = C.CUSTOMERNUMBER;",
    conn)

group_basic_df.head()

,orderNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber,orderNumber,productCode,quantityOrdered,...,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,10100,2003-01-06 00:00:00,2003-01-13 00:00:00,2003-01-10 00:00:00,Shipped,None,363,10100,S18_1749,30,...,Dorothy,6035558647,2304 Long Airport Avenue,None,Nashua,NH,62005,USA,1216.0,114200.0
1,10100,2003-01-06 00:00:00,2003-01-13 00:00:00,2003-01-10 00:00:00,Shipped,None,363,10100,S18_2248,50,...,Dorothy,6035558647,2304 Long Airport Avenue,None,Nashua,NH,62005,USA,1216.0,114200.0
2,10100,2003-01-06 00:00:00,2003-01-13 00:00:00,2003-01-10 00:00:00,Shipped,None,363,10100,S18_4409,22,...,Dorothy,6035558647,2304 Long Airport Avenue,None,Nashua,NH,62005,USA,1216.0,114200.0
3,10100,2003-01-06 00:00:00,2003-01-13 00:00:00,2003-01-10 00:00:00,Shipped,None,363,10100,S24_3969,49,...,Dorothy,6035558647,2304 Long Airport Avenue,None,Nashua,NH,62005,USA,1216.0,114200.0
4,10101,2003-01-09 00:00:00,2003-01-18 00:00:00,2003-01-11 00:00:00,Shipped,Check on availability.,128,10101,S18_2325,25,...,Roland,+49 69 66 90 2555,Lyonerstr. 34,None,Frankfurt,None,60528,Germany,1504.0,59700.0


In [42]:
group_basic_df.columns

Index(['orderNumber', 'orderDate', 'requiredDate', 'shippedDate', 'status',
       'comments', 'customerNumber', 'orderNumber', 'productCode',
       'quantityOrdered', 'priceEach', 'orderLineNumber', 'customerNumber',
       'customerName', 'contactLastName', 'contactFirstName', 'phone',
       'addressLine1', 'addressLine2', 'city', 'state', 'postalCode',
       'country', 'salesRepEmployeeNumber', 'creditLimit'],
      dtype='object')

In [45]:
group_simple_df = pd.read_sql_query(
    "SELECT C.COUNTRY, C.CITY,\
    SUM(PRICEEACH*QUANTITYORDERED) SALES\
    FROM ORDERS A\
    LEFT JOIN ORDERDETAILS B\
    ON A.ORDERNUMBER = B.ORDERNUMBER\
    LEFT JOIN CUSTOMERS C\
    ON A.CUSTOMERNUMBER = C.CUSTOMERNUMBER\
    GROUP BY 1,2\
    ORDER BY 1,2;",
    conn)
group_simple_df.head(10)

,country,city,SALES
0,Australia,Chatswood,133907.12
1,Australia,Glen Waverly,55866.02
2,Australia,Melbourne,180585.07
3,Australia,North Sydney,137034.22
4,Australia,South Brisbane,55190.16
5,Austria,Graz,51059.99
6,Austria,Salzburg,137480.07
7,Belgium,Bruxelles,70851.58
8,Belgium,Charleroi,29217.18
9,Canada,Montréal,68977.67


In [46]:
# 북미 지역과 아닌 지역 매출액 비교

north_am_df = pd.read_sql_query(
    "SELECT CASE WHEN COUNTRY IN ('USA', 'Canada') THEN 'North America' ELSE 'Others' END COUNTRY_GRP,\
    SUM(PRICEEACH*QUANTITYORDERED) SALES\
    FROM ORDERS A\
    LEFT JOIN ORDERDETAILS B\
    ON A.ORDERNUMBER = B.ORDERNUMBER\
    LEFT JOIN CUSTOMERS C\
    ON A.CUSTOMERNUMBER = C.CUSTOMERNUMBER\
    GROUP BY 1\
    ORDER BY 2 DESC;",
    conn)
north_am_df.head(10)

,COUNTRY_GRP,SALES
0,Others,6124998.70
1,North America,3479191.91


In [57]:
# 매출액 Top 5 국가

c = conn.cursor()

c.execute(
    "CREATE TABLE IF NOT EXISTS STAT AS\
    SELECT C.COUNTRY,\
    SUM(PRICEEACH*QUANTITYORDERED) SALES\
    FROM ORDERS A\
    LEFT JOIN ORDERDETAILS B\
    ON A.ORDERNUMBER = B.ORDERNUMBER\
    LEFT JOIN CUSTOMERS C\
    ON A.CUSTOMERNUMBER = C.CUSTOMERNUMBER\
    GROUP BY 1\
    ORDER BY 2 DESC;"
)

c.execute(
    "SELECT * FROM STAT;"
    )

c.fetchall()

[('USA', 3273280.0500000035),
 ('Spain', 1099389.09),
 ('France', 1007374.0200000006),
 ('Australia', 562582.5900000001),
 ('New Zealand', 476847.0099999999),
 ('UK', 436947.4400000001),
 ('Italy', 360616.8099999998),
 ('Finland', 295149.3499999999),
 ('Singapore', 263997.77999999985),
 ('Denmark', 218994.91999999998),
 ('Canada', 205911.86),
 ('Germany', 196470.99),
 ('Austria', 188540.06),
 ('Sweden', 187638.34999999998),
 ('Japan', 167909.95000000004),
 ('Norway  ', 166621.50999999998),
 ('Switzerland', 108777.92),
 ('Norway', 104224.78999999996),
 ('Belgium', 100068.75999999998),
 ('Philippines', 87468.29999999999),
 ('Ireland', 49898.270000000004),
 ('Hong Kong', 45480.79)]

In [61]:
c.execute(
    "SELECT COUNTRY, SALES,\
    DENSE_RANK() OVER(ORDER BY SALES DESC) RNK\
    FROM STAT;"
    )

c.fetchall()

[('USA', 3273280.0500000035, 1),
 ('Spain', 1099389.09, 2),
 ('France', 1007374.0200000006, 3),
 ('Australia', 562582.5900000001, 4),
 ('New Zealand', 476847.0099999999, 5),
 ('UK', 436947.4400000001, 6),
 ('Italy', 360616.8099999998, 7),
 ('Finland', 295149.3499999999, 8),
 ('Singapore', 263997.77999999985, 9),
 ('Denmark', 218994.91999999998, 10),
 ('Canada', 205911.86, 11),
 ('Germany', 196470.99, 12),
 ('Austria', 188540.06, 13),
 ('Sweden', 187638.34999999998, 14),
 ('Japan', 167909.95000000004, 15),
 ('Norway  ', 166621.50999999998, 16),
 ('Switzerland', 108777.92, 17),
 ('Norway', 104224.78999999996, 18),
 ('Belgium', 100068.75999999998, 19),
 ('Philippines', 87468.29999999999, 20),
 ('Ireland', 49898.270000000004, 21),
 ('Hong Kong', 45480.79, 22)]

In [63]:
c.execute(
    "CREATE TABLE IF NOT EXISTS STAT_RNK AS\
    SELECT COUNTRY, SALES,\
    DENSE_RANK() OVER(ORDER BY SALES DESC) RNK\
    FROM STAT;"
    )

c.execute(
    "SELECT * FROM STAT_RNK\
    WHERE RNK BETWEEN 1 AND 5;"
)

c.fetchall()

[('USA', 3273280.0500000035, 1),
 ('Spain', 1099389.09, 2),
 ('France', 1007374.0200000006, 3),
 ('Australia', 562582.5900000001, 4),
 ('New Zealand', 476847.0099999999, 5)]

In [68]:
top_5_country = pd.read_sql_query(
    "SELECT *\
    FROM(\
      SELECT COUNTRY, SALES,\
      DENSE_RANK() OVER(ORDER BY SALES DESC) RNK\
      FROM(\
        SELECT C.COUNTRY,\
        SUM(PRICEEACH*QUANTITYORDERED) SALES\
        FROM ORDERS A\
        LEFT JOIN ORDERDETAILS B\
        ON A.ORDERNUMBER = B.ORDERNUMBER\
        LEFT JOIN CUSTOMERS C\
        ON A.CUSTOMERNUMBER = C.CUSTOMERNUMBER\
        GROUP BY 1\
        ) A\
      ) A\
      WHERE RNK <= 5;",
      conn)

top_5_country.head()

,COUNTRY,SALES,RNK
0,USA,3273280.05,1
1,Spain,1099389.09,2
2,France,1007374.02,3
3,Australia,562582.59,4
4,New Zealand,476847.01,5


In [105]:
# 국가별 재구매율

# sqlite3 에서는 정수에 대한 나눗셈 연산은 정수로만 결과 반환
# 해결법 : CAST(a as REAL) or a + 0.00 으로 계산
# 문자열과 정수형 계산시 데이터 타입이 변함 다시 문자 타입으로 변경 필요

re_buy_df = pd.read_sql_query(
    "SELECT C.COUNTRY,\
    SUBSTR(A.ORDERDATE,1,4) YY,\
    COUNT(DISTINCT A.CUSTOMERNUMBER) Buy_1,\
    COUNT(DISTINCT B.CUSTOMERNUMBER) Buy_2,\
    (COUNT(DISTINCT B.CUSTOMERNUMBER) + 0.00) / (COUNT(DISTINCT A.CUSTOMERNUMBER) +0.00) RETENTION_RATE\
    FROM ORDERS A\
    LEFT JOIN ORDERS B\
    ON A.CUSTOMERNUMBER = B.CUSTOMERNUMBER AND\
    SUBSTR(A.ORDERDATE,1,4) = CAST(SUBSTR(B.ORDERDATE,1,4) - 1 AS TEXT)\
    LEFT JOIN CUSTOMERS C\
    ON A.CUSTOMERNUMBER = C.CUSTOMERNUMBER\
    GROUP BY 1,2;",
    conn
    )

re_buy_df.head()

,country,YY,Buy_1,Buy_2,RETENTION_RATE
0,Australia,2003,5,3,0.600000
1,Australia,2004,3,2,0.666667
2,Australia,2005,4,0,0.000000
3,Austria,2003,2,1,0.500000
4,Austria,2004,1,1,1.000000


# Best Seller

In [112]:
# 미국 시장 Top 5
c.execute(
    "CREATE TABLE IF NOT EXISTS PRODUCT_SALES AS\
    SELECT D.PRODUCTNAME,\
    SUM(QUANTITYORDERED*PRICEEACH) SALES\
    FROM ORDERS A\
    LEFT JOIN CUSTOMERS B\
    ON A.CUSTOMERNUMBER = B.CUSTOMERNUMBER\
    LEFT JOIN ORDERDETAILS C\
    ON A.ORDERNUMBER = C.ORDERNUMBER\
    LEFT JOIN PRODUCTS D\
    ON C.PRODUCTCODE = D.PRODUCTCODE\
    WHERE B.COUNTRY = 'USA'\
    GROUP BY 1;" 
    )

usa_top5_df = pd.read_sql_query(
    "SELECT *\
    FROM(\
      SELECT *,\
      ROW_NUMBER() OVER(ORDER BY SALES DESC) RNK\
      FROM PRODUCT_SALES\
      ) A\
    WHERE RNK <= 5\
    ORDER BY RNK",
    conn)

usa_top5_df.head()

,productName,SALES,RNK
0,1952 Alpine Renault 1300,78860.11,1
1,1992 Ferrari 360 Spider red,76011.71,2
2,2003 Harley-Davidson Eagle Drag Bike,74524.35,3
3,1998 Chrysler Plymouth Prowler,66599.93,4
4,2002 Suzuki XREO,66150.36,5


# Churn Rate 고객 이탈

In [170]:
churn_df = pd.read_sql_query(
            "SELECT *,\
            CASE WHEN DIFF >= 90 THEN 'CHURN' ELSE 'NON-CHURN' END CHURN_TYPE\
            FROM\
              (SELECT CUSTOMERNUMBER,\
              MX_ORDER,\
              '2005-06-01' END_POINT,\
              julianday('2005-06-01') -  julianday(MX_ORDER) DIFF\
            FROM\
              (SELECT CUSTOMERNUMBER,\
              MAX(SUBSTR(ORDERDATE,1,10)) MX_ORDER\
              FROM ORDERS\
              GROUP\
              BY 1) BASE) BASE;",conn)

churn_df.head(10)

,CUSTOMERNUMBER,MX_ORDER,END_POINT,DIFF,CHURN_TYPE
0,103,2004-11-25,2005-06-01,188.0,CHURN
1,112,2004-11-29,2005-06-01,184.0,CHURN
2,114,2004-11-29,2005-06-01,184.0,CHURN
3,119,2005-05-31,2005-06-01,1.0,NON-CHURN
4,121,2004-11-05,2005-06-01,208.0,CHURN
5,124,2005-05-29,2005-06-01,3.0,NON-CHURN
6,128,2004-11-05,2005-06-01,208.0,CHURN
7,129,2004-11-18,2005-06-01,195.0,CHURN
8,131,2004-11-15,2005-06-01,198.0,CHURN
9,141,2005-05-31,2005-06-01,1.0,NON-CHURN


In [172]:
churn_90 = pd.read_sql_query(
            "SELECT CASE WHEN DIFF >= 90 THEN 'CHURN' ELSE 'NON-CHURN' END CHURN_TYPE,\
            COUNT(DISTINCT CUSTOMERNUMBER) N_CUS\
            FROM\
              (SELECT CUSTOMERNUMBER,\
              MX_ORDER,\
              '2005-06-01' END_POINT,\
              julianday('2005-06-01') -  julianday(MX_ORDER) DIFF\
            FROM\
              (SELECT CUSTOMERNUMBER,\
              MAX(SUBSTR(ORDERDATE,1,10)) MX_ORDER\
              FROM ORDERS\
              GROUP\
              BY 1) BASE) BASE\
              GROUP BY 1;",conn)

churn_90

,CHURN_TYPE,N_CUS
0,CHURN,69
1,NON-CHURN,29


In [ ]:
c.execute(
    "CREATE TABLE CHURN_LIST AS\
    SELECT CASE WHEN DIFF >= 90 THEN 'CHURN' ELSE 'NON-CHURN' END CHURN_TYPE,\
    CUSTOMERNUMBER\
    FROM\
      (SELECT CUSTOMERNUMBER,\
      MX_ORDER,\
      '2005-06-01' END_POINT,\
      julianday('2005-06-01') -  julianday(MX_ORDER) DIFF\
    FROM\
      (SELECT CUSTOMERNUMBER,\
      MAX(SUBSTR(ORDERDATE,1,10)) MX_ORDER\
      FROM ORDERS\
      GROUP\
      BY 1) BASE) BASE;"
)

c.execute(
    "SELECT * FROM CHURN_LIST;"
    )

c.fetchall()

In [176]:
pd.read_sql_query(
    "SELECT C.PRODUCTLINE,\
    COUNT(DISTINCT B.CUSTOMERNUMBER) BU\
    FROM ORDERDETAILS A\
    LEFT JOIN ORDERS B\
    ON A.ORDERNUMBER = B.ORDERNUMBER\
    LEFT JOIN PRODUCTS C\
    ON A.PRODUCTCODE = C.PRODUCTCODE\
    GROUP BY 1;",
    conn
)

,productLine,BU
0,Classic Cars,94
1,Motorcycles,55
2,Planes,52
3,Ships,51
4,Trains,36
5,Trucks and Buses,50
6,Vintage Cars,90


In [180]:
churn_buy_df =   pd.read_sql_query(
                  "SELECT D.CHURN_TYPE, C.PRODUCTLINE,\
                  COUNT(DISTINCT B.CUSTOMERNUMBER) BU\
                  FROM ORDERDETAILS A\
                  LEFT JOIN ORDERS B\
                  ON A.ORDERNUMBER = B.ORDERNUMBER\
                  LEFT JOIN PRODUCTS C\
                  ON A.PRODUCTCODE = C.PRODUCTCODE\
                  LEFT JOIN CHURN_LIST D\
                  ON B.CUSTOMERNUMBER = D.CUSTOMERNUMBER\
                  GROUP BY 1,2\
                  ORDER BY 1,3 DESC;",
                  conn)

churn_buy_df

,CHURN_TYPE,productLine,BU
0,CHURN,Classic Cars,66
1,CHURN,Vintage Cars,63
2,CHURN,Trucks and Buses,36
3,CHURN,Planes,35
4,CHURN,Motorcycles,34
5,CHURN,Ships,33
6,CHURN,Trains,23
7,NON-CHURN,Classic Cars,28
8,NON-CHURN,Vintage Cars,27
9,NON-CHURN,Motorcycles,21


In [186]:
conn.close()